# Single Phase Flow Examples


## Example 1. Single well in the middle of a reservoir.

In this example we have a reservoir with four layers with a single producer in the middle. The initial pressure of the reservoir is 3000 psi and the bottom hole pressure of the well is 2500 psi.

In [1]:
from modules.simulator.simulator import *
from modules.simulator.two_phase_flow import *
from modules.simulator.simplots import *
from modules.simulator.units import UnitRegistry
import numpy as np

In [8]:
u = UnitRegistry()
# GRID
Nx, Ny, Nz = np.array([19, 19, 4])
Sx, Sy, Sz = np.array([1, 1, 1]) * u.meter
depth = np.hstack((Sz * 3 * np.ones([Nx * Ny, ]),
                   Sz * 2 * np.ones([Nx * Ny, ]),
                   Sz * 1 * np.ones([Nx * Ny, ]),
                   Sz * 0 * np.ones([Nx * Ny, ])))
grid = uniformCartesianGrid(Nx, Ny, Nz, Sx, Sy, Sz, depth)

# ROCK
poro = np.random.random_sample(grid.cellnumber)
perm = 5 * np.random.random_sample(grid.cellnumber) * u.milli * u.darcy
cr = 3E-6 / u.psi
porofunc = lambda p: poro * np.exp(cr * (p - 2800 * u.psi))
rock = Rock(perm, poro, cr, porofunc)

# FLUID
cf = 5E-5 / u.psi
miu = lambda p:  1.0 * np.exp(5E-5 * (p/u.psi -2800)) * u.centi * u.poise
rho = lambda p: 800 * np.exp(cf * (p - 15 * u.psi) )  * u.kilogram/ u.meter**3
fvf = lambda p: 1.0 * np.exp(- cf * (p - 15 * u.psi) )   # adimensional
fluid = singleFluid(miu, rho, fvf, cf)

# SCHEDULE
timesteps = 0.1 * np.ones(10) *u.day
sch = Schedule(timesteps)

# Create empty boundary and source term
boundary1 = Boundary()
source = np.zeros([grid.cellnumber, 1])
# Set initial conditions
p_init = 3000 * u.psi * np.ones([grid.cellnumber, ])

# --- WELLS
wells = Wells(grid, rock, fluid)
wells.add_vertical_well(0.35 * u.feet, 179, 2800 * u.psi, 0, 'Producer 1' )

# --- Initilize simulator
LCM = LaggingCoefficients(
    grid, rock, fluid, wells, source, p_init, boundary1, gravity=True)

# Run simulation
r, well_solution, sch = LCM.solve(sch, max_inner_iter = 2, tol = 1E-6, ATS = False)

Solving timestep 1 / 10
	 Inner loop 1 / 2
	 Inner loop 2 / 2
Solving timestep 2 / 10
	 Inner loop 1 / 2
	 Inner loop 2 / 2
Solving timestep 3 / 10
	 Inner loop 1 / 2
	 Inner loop 2 / 2
Solving timestep 4 / 10
	 Inner loop 1 / 2
	 Inner loop 2 / 2
Solving timestep 5 / 10
	 Inner loop 1 / 2
	 Inner loop 2 / 2
Solving timestep 6 / 10
	 Inner loop 1 / 2
	 Inner loop 2 / 2
Solving timestep 7 / 10
	 Inner loop 1 / 2
	 Inner loop 2 / 2
Solving timestep 8 / 10
	 Inner loop 1 / 2
	 Inner loop 2 / 2
Solving timestep 9 / 10
	 Inner loop 1 / 2
	 Inner loop 2 / 2
Solving timestep 10 / 10
	 Inner loop 1 / 2
	 Inner loop 2 / 2


In [9]:
# Set path to images folder
imgpath = ".\\images"
# The pressure solution is in pascals, we need to convert it to psi
r = r / u.psi

acum_sum = np.cumsum(sch.timesteps) / u.day
acum_sum = np.hstack((0, acum_sum))
for k in np.arange(0, acum_sum.size, 1): 
    plotCellValues2D(grid, r[:, k], 'magma', np.min(r), np.max(r), 
      title='{:.3f} days'.format(acum_sum[k]), filename = '{}\\timestep_{}'.format(imgpath, k))

C:\Users\Jaime\Anaconda3\lib\site-packages\matplotlib\figure.py:1718: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "


<img src=".\images\timestep_1.png">
<img src=".\images\timestep_2.png">
<img src=".\images\timestep_3.png">
<img src=".\images\timestep_4.png">
<img src=".\images\timestep_10.png">


## Example 2. Two injectors one producer.


## Solving with Lagging Coefficients Method

In [10]:
plotRate(wells, sch, imgpath)

In [11]:
timesteps = np.ones(20) * 1 * u.day
sch = Schedule(timesteps)

FIM  = ImplicitAnalytic(
    grid, rock, fluid, wells, source, p_init, boundary1, gravity=True)

r, well_solution = FIM.solve(sch)

Solving timestep 1  / 20
 	 Newton-Raphson solver :  1/10. Error: 5.06E-03
 	 Newton-Raphson solver :  2/10. Error: 3.28E-07
 	 Newton-Raphson solver :  3/10. Error: 1.07E-11
 	 Newton-Raphson solver :  4/10. Error: 1.15E-15
Solving timestep 2  / 20
 	 Newton-Raphson solver :  1/10. Error: 3.98E-03
 	 Newton-Raphson solver :  2/10. Error: 2.97E-07
 	 Newton-Raphson solver :  3/10. Error: 1.08E-11
 	 Newton-Raphson solver :  4/10. Error: 1.10E-15
Solving timestep 3  / 20
 	 Newton-Raphson solver :  1/10. Error: 3.73E-03
 	 Newton-Raphson solver :  2/10. Error: 2.78E-07
 	 Newton-Raphson solver :  3/10. Error: 1.00E-11
 	 Newton-Raphson solver :  4/10. Error: 1.57E-15
Solving timestep 4  / 20
 	 Newton-Raphson solver :  1/10. Error: 3.51E-03
 	 Newton-Raphson solver :  2/10. Error: 2.59E-07
 	 Newton-Raphson solver :  3/10. Error: 9.15E-12
 	 Newton-Raphson solver :  4/10. Error: 1.12E-15
Solving timestep 5  / 20
 	 Newton-Raphson solver :  1/10. Error: 3.30E-03
 	 Newton-Raphson solver 

In [12]:
r = r / u.psi

acum_sum = np.cumsum(sch.timesteps) / u.day
acum_sum = np.hstack((0, acum_sum))
for k in np.arange(0, acum_sum.size, 1): 
    plotCellValues2D(grid, r[:, k], 'magma', np.min(r), np.max(r), 
      title='{:.3f} days'.format(acum_sum[k]), filename = '{}\\fim_timestep_{}'.format(imgpath, k))

C:\Users\Jaime\Anaconda3\lib\site-packages\matplotlib\figure.py:1718: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "


## Example 2. One injector and two producers.

## Visualizing the solution



### Pressure distribution

### Well rate


In [14]:
well_solution


[{'T0': 2.6082085296888876e-12,
  'bhp': 19305328.0,
  'location': 179,
  'name': 'Producer 1',
  'neighbor_cells': array([[ 178.,    1.],
         [ 180.,    1.],
         [ 160.,    1.],
         [ 198.,    1.],
         [ 161.,    0.],
         [ 159.,    0.],
         [ 197.,    0.],
         [ 199.,    0.]]),
  'rate': [0,
   3.6543847382944662e-05,
   0,
   3.682221179766157e-05,
   0,
   3.6879157305828672e-05,
   0,
   3.6895059686073878e-05,
   0,
   3.6900126975584929e-05,
   0,
   3.6901890187387512e-05,
   0,
   3.6902562703001958e-05,
   0,
   3.6902847050535982e-05,
   0,
   3.6902980597274909e-05,
   0,
   3.6903049483650341e-05,
   0,
   3.6543847382944662e-05,
   6.6779129589280235e-06,
   6.6776999293847585e-06,
   6.6776999313162062e-06,
   6.6776999313163044e-06,
   6.1289259452497742e-06,
   6.1288619907199478e-06,
   6.1288619895396938e-06,
   6.1288619895396938e-06,
   5.7334810450415927e-06,
   5.7334553164692143e-06,
   5.7334553145565241e-06,
   5.733455314556

In [7]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

# I am using the style sheet found here  Lorena Barba https://github.com/barbagroup/CFDPython/blob/master/styles/custom.css